In [14]:
import pickle
import joblib
import os 

import pandas as pd
import numpy as np

from numpy import mean, std
from joblib import dump, load
from train_test_split import train_test_split_custom
from datetime import timedelta
from xgboost import XGBRegressor
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder, Normalizer

# HERE YOU HAVE TO CHOOSE BPI_Challenge_2012 or BPI_Challenge_2017
chosed_dataset = 'BPI_Challenge_2017'

# RFC for next event prediction

In [15]:
# extract the 1-3 lags
df = pd.read_csv(f'../data/{chosed_dataset}_naive.csv')

df['concept:name - lag_1'] = df.groupby('case:concept:name')['concept:name'].shift(1)
df['concept:name - lag_2'] = df.groupby('case:concept:name')['concept:name'].shift(2)
df['concept:name - lag_3'] = df.groupby('case:concept:name')['concept:name'].shift(3)

# define target
df['next concept:name'] = df.groupby('case:concept:name')['concept:name'].shift(-1)

# # Split the DataFrame 
# df = df.iloc[:1000]

df

,Unnamed: 0.1,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,OfferID,start_timestamp,time:timestamp diff,next time:timestamp,next time:timestamp naive,next concept:name naive,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,next concept:name
0,0,0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304,Existing loan takeover,...,NaN,2016-01-01 09:51:15.304000+00:00,NaN,2016-01-01 09:51:15.352,2016-01-01 09:51:15.344,A_Submitted,NaN,NaN,NaN,A_Submitted
1,1,1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352,Existing loan takeover,...,NaN,2016-01-01 09:51:15.352000+00:00,0 days 00:00:00.048000,2016-01-01 09:51:15.774,2016-01-01 09:51:15.652,W_Handle leads,A_Create Application,NaN,NaN,W_Handle leads
2,2,2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774,Existing loan takeover,...,NaN,2016-01-01 09:51:15.774000+00:00,0 days 00:00:00.422000,2016-01-01 09:52:36.392,2016-01-01 09:51:49.854,W_Handle leads,A_Submitted,A_Create Application,NaN,W_Handle leads
3,3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,...,NaN,2016-01-01 09:52:36.392000+00:00,0 days 00:01:20.618000,2016-01-01 09:52:36.403,2016-01-01 09:53:10.472,W_Complete application,W_Handle leads,A_Submitted,A_Create Application,W_Complete application
4,4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,...,NaN,2016-01-01 09:52:36.403000+00:00,0 days 00:00:00.011000,2016-01-01 09:52:36.413,2016-01-01 09:52:40.113,A_Concept,W_Handle leads,W_Handle leads,A_Submitted,A_Concept
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201085,1201085,1071364,statechange,User_131,A_Validating,Application,ApplState_1576288280,complete,2017-02-01 14:00:30.347,Existing loan takeover,...,NaN,2017-02-01 14:00:30.347000+00:00,0 days 00:00:00.072000,2017-02-01 14:01:24.191,2017-02-01 14:00:56.427,W_Call incomplete files,W_Validate application,W_Validate application,W_Call incomplete files,W_Validate application
1201086,1201086,1071365,Released,User_131,W_Validate application,Workflow,Workitem_997581455,suspend,2017-02-01 14:01:24.191,Existing loan takeover,...,NaN,2017-02-01 14:01:24.191000+00:00,0 days 00:00:53.844000,NaN,2017-02-01 14:01:25.031,NaN,A_Validating,W_Validate application,W_Validate application,NaN
1201087,1201087,1169224,Released,User_56,W_Call incomplete files,Workflow,Workitem_1421607593,suspend,2017-02-01 14:09:03.331,"Other, see explanation",...,NaN,2017-02-01 14:09:03.331000+00:00,0 days 00:18:06.767000,NaN,2017-02-01 14:10:13.841,NaN,W_Call incomplete files,W_Call incomplete files,A_Incomplete,NaN
1201088,1201088,1191694,Obtained,User_100,W_Validate application,Workflow,Workitem_1317562192,resume,2017-02-01 14:10:52.793,Existing loan takeover,...,NaN,2017-02-01 14:10:52.793000+00:00,0 days 00:16:49.249000,2017-02-01 14:11:03.499,2017-02-01 14:10:53.633,W_Call incomplete files,W_Validate application,A_Validating,W_Validate application,W_Validate application


In [16]:
df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'Action', 'org:resource', 'concept:name',
       'EventOrigin', 'EventID', 'lifecycle:transition', 'time:timestamp',
       'case:LoanGoal', 'case:ApplicationType', 'case:concept:name',
       'case:RequestedAmount', 'FirstWithdrawalAmount', 'NumberOfTerms',
       'Accepted', 'MonthlyCost', 'Selected', 'CreditScore', 'OfferedAmount',
       'OfferID', 'start_timestamp', 'time:timestamp diff',
       'next time:timestamp', 'next time:timestamp naive',
       'next concept:name naive', 'concept:name - lag_1',
       'concept:name - lag_2', 'concept:name - lag_3', 'next concept:name'],
      dtype='object')

In [17]:
# Prepare data

df_train, df_test = train_test_split_custom(df=df, test_size=0.2, lags=True)

columns = ['concept:name' , 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3', 'next concept:name']
label_encoders = {}
for column in columns:
        label_encoder = LabelEncoder()
        df_test[column] = label_encoder.fit_transform(df_test[column])
        df_train[column] = label_encoder.fit_transform(df_train[column])
        df[column] = label_encoder.fit_transform(df[column])
        label_encoders[column] = label_encoder

X_train = df_train[['concept:name', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3']]
X_test = df_test[['concept:name', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3']]

y_train = df_train[['next concept:name']]
y_test = df_test[['next concept:name']]

print(f"""
    inputs: {[col for col in X_test.columns]} \n
    target: {[col for col in y_test.columns]}
""")

if not os.path.exists('../model_weights/random_forest.pkl'):

    rf_clf = RandomForestClassifier(n_jobs=-1)

    # Define the parameter grid for hyperparameter tuning
    param_grid = {
        'n_estimators': [21, 22, 23],
        'max_depth': [18, 19, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # Perform nested cross-validation
    inner_cv = KFold(n_splits=5, shuffle=False, random_state=None)  # 5-fold inner cross-validation

    grid_search = GridSearchCV(estimator=rf_clf, param_grid=param_grid, scoring='accuracy', cv=inner_cv)
    grid_search.fit(X_train, y_train.values.ravel())

    # Get the best model
    best_model = grid_search.best_estimator_

    # Print the results

    print(f"""
        Next label prediction:\n
        Score on the test set {best_model.score(X_test, y_test)}
        Best parameters: {grid_search.best_params_}
    """)

    # Save the best model to a file using pickle
    # Create the folder for model weights
    os.makedirs('../model_weights', exist_ok=True)
    model_filename = '../model_weights/random_forest.pkl'

    with open(model_filename, 'wb') as model_file:
        pickle.dump(best_model, model_file)
        
else: 
    with open('../model_weights/random_forest.pkl', 'rb') as f:
        best_model = pickle.load(f)

# Make predictions on the dataset for adding new column
df['next concept:name rfc'] = label_encoder.inverse_transform(best_model.predict(df[['concept:name', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3']]))

for column in columns:
        df[column] = label_encoders[column].inverse_transform(df[column])

df

Train set ends with 2016-10-27 09:12:04.286
Test set starts with: 2016-10-27 09:26:15.278

    inputs: ['concept:name', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3'] 

    target: ['next concept:name']



,Unnamed: 0.1,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,...,start_timestamp,time:timestamp diff,next time:timestamp,next time:timestamp naive,next concept:name naive,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,next concept:name,next concept:name rfc
0,0,0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304,Existing loan takeover,...,2016-01-01 09:51:15.304000+00:00,NaN,2016-01-01 09:51:15.352,2016-01-01 09:51:15.344,A_Submitted,NaN,NaN,NaN,A_Submitted,A_Submitted
1,1,1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352,Existing loan takeover,...,2016-01-01 09:51:15.352000+00:00,0 days 00:00:00.048000,2016-01-01 09:51:15.774,2016-01-01 09:51:15.652,W_Handle leads,A_Create Application,NaN,NaN,W_Handle leads,W_Handle leads
2,2,2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774,Existing loan takeover,...,2016-01-01 09:51:15.774000+00:00,0 days 00:00:00.422000,2016-01-01 09:52:36.392,2016-01-01 09:51:49.854,W_Handle leads,A_Submitted,A_Create Application,NaN,W_Handle leads,W_Handle leads
3,3,3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,...,2016-01-01 09:52:36.392000+00:00,0 days 00:01:20.618000,2016-01-01 09:52:36.403,2016-01-01 09:53:10.472,W_Complete application,W_Handle leads,A_Submitted,A_Create Application,W_Complete application,W_Complete application
4,4,4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,...,2016-01-01 09:52:36.403000+00:00,0 days 00:00:00.011000,2016-01-01 09:52:36.413,2016-01-01 09:52:40.113,A_Concept,W_Handle leads,W_Handle leads,A_Submitted,A_Concept,A_Concept
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201085,1201085,1071364,statechange,User_131,A_Validating,Application,ApplState_1576288280,complete,2017-02-01 14:00:30.347,Existing loan takeover,...,2017-02-01 14:00:30.347000+00:00,0 days 00:00:00.072000,2017-02-01 14:01:24.191,2017-02-01 14:00:56.427,W_Call incomplete files,W_Validate application,W_Validate application,W_Call incomplete files,W_Validate application,W_Shortened completion
1201086,1201086,1071365,Released,User_131,W_Validate application,Workflow,Workitem_997581455,suspend,2017-02-01 14:01:24.191,Existing loan takeover,...,2017-02-01 14:01:24.191000+00:00,0 days 00:00:53.844000,NaN,2017-02-01 14:01:25.031,NaN,A_Validating,W_Validate application,W_Validate application,NaN,W_Shortened completion
1201087,1201087,1169224,Released,User_56,W_Call incomplete files,Workflow,Workitem_1421607593,suspend,2017-02-01 14:09:03.331,"Other, see explanation",...,2017-02-01 14:09:03.331000+00:00,0 days 00:18:06.767000,NaN,2017-02-01 14:10:13.841,NaN,W_Call incomplete files,W_Call incomplete files,A_Incomplete,NaN,W_Call incomplete files
1201088,1201088,1191694,Obtained,User_100,W_Validate application,Workflow,Workitem_1317562192,resume,2017-02-01 14:10:52.793,Existing loan takeover,...,2017-02-01 14:10:52.793000+00:00,0 days 00:16:49.249000,2017-02-01 14:11:03.499,2017-02-01 14:10:53.633,W_Call incomplete files,W_Validate application,A_Validating,W_Validate application,W_Validate application,W_Shortened completion


In [18]:
df.to_csv(f'../data/{chosed_dataset}_rfc_xgboost.csv')

# XGBoost for time till next event prediction

In [19]:
df = pd.read_csv(f'../data/{chosed_dataset}_rfc_xgboost.csv')

# counting elapsed time
df['elapsed time:timestamp'] = df['time:timestamp diff'].shift(-1) 
df['elapsed time:timestamp'] = pd.to_timedelta(df['elapsed time:timestamp'])
df['elapsed time:timestamp'] = df['elapsed time:timestamp'].apply(lambda x: x.total_seconds()).fillna(-0.01)
df['time:timestamp'] = pd.to_datetime(df['time:timestamp'])
df['CreditScore'] = df['CreditScore'].fillna(0)

# lags for time between events in past 

df['elapsed time:timestamp - lag_1'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(1).fillna(-0.00000001)
df['elapsed time:timestamp - lag_2'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(2).fillna(-0.00000001)
df['elapsed time:timestamp - lag_3'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(3).fillna(-0.00000001)
df['elapsed time:timestamp - lag_4'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(4).fillna(-0.00000001)
df['elapsed time:timestamp - lag_5'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(5).fillna(-0.00000001)
df['elapsed time:timestamp - lag_6'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(6).fillna(-0.00000001)
df['elapsed time:timestamp - lag_7'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(7).fillna(-0.00000001)
df['elapsed time:timestamp - lag_8'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(8).fillna(-0.00000001)
df['elapsed time:timestamp - lag_9'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(9).fillna(-0.00000001)
df['elapsed time:timestamp - lag_10'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(10).fillna(-0.00000001)
df['elapsed time:timestamp - lag_11'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(11).fillna(-0.00000001)
df['elapsed time:timestamp - lag_12'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(12).fillna(-0.00000001)
df['elapsed time:timestamp - lag_13'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(13).fillna(-0.00000001)
df['elapsed time:timestamp - lag_14'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(14).fillna(-0.00000001)
df['elapsed time:timestamp - lag_15'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(15).fillna(-0.00000001)
df['elapsed time:timestamp - lag_16'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(16).fillna(-0.00000001)
df['elapsed time:timestamp - lag_17'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(17).fillna(-0.00000001)
df['elapsed time:timestamp - lag_18'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(18).fillna(-0.00000001)
df['elapsed time:timestamp - lag_19'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(19).fillna(-0.00000001)
df['elapsed time:timestamp - lag_20'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(20).fillna(-0.00000001)
df['elapsed time:timestamp - lag_21'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(21).fillna(-0.00000001)
df['elapsed time:timestamp - lag_22'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(22).fillna(-0.00000001)
df['elapsed time:timestamp - lag_23'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(23).fillna(-0.00000001)
df['elapsed time:timestamp - lag_24'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(24).fillna(-0.00000001)
df['elapsed time:timestamp - lag_25'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(25).fillna(-0.00000001)
df['elapsed time:timestamp - lag_26'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(26).fillna(-0.00000001)
df['elapsed time:timestamp - lag_27'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(27).fillna(-0.00000001)
df['elapsed time:timestamp - lag_28'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(28).fillna(-0.00000001)
df['elapsed time:timestamp - lag_29'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(29).fillna(-0.00000001)
df['elapsed time:timestamp - lag_30'] = df.groupby(by='case:concept:name')['elapsed time:timestamp'].shift(30).fillna(-0.00000001)

# preprocess the columns before fitting
preprocessors = {}
for column in df.columns:
    if column == 'time:timestamp':
        df['year'] = df['time:timestamp'].dt.year
        df['month'] = df['time:timestamp'].dt.month
        df['day'] = df['time:timestamp'].dt.day
        df['hour'] = df['time:timestamp'].dt.hour

    elif column in ['concept:name', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3', 'next concept:name rfc', 'org:resource']:
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column])
        preprocessors[column] = le

    else:
        continue
    
# split the data on train and test dataframes 
df_train, df_test = train_test_split_custom(df=df, lags=False, test_size=0.2)

# define the input and outputs for the model before put in the train 

X = df[['concept:name',  # this X is for making prediction on the dataframe for saving results in the csv
                    'concept:name - lag_1', 
                    'concept:name - lag_2', 
                    'concept:name - lag_3', 
                    'next concept:name rfc', 
                    'year', 
                    'month', 
                    'day', 
                    'hour',
                    'CreditScore',
                    'org:resource',
                    'elapsed time:timestamp - lag_1', 
                    'elapsed time:timestamp - lag_2', 
                    'elapsed time:timestamp - lag_3',
                    'elapsed time:timestamp - lag_4',
                    'elapsed time:timestamp - lag_5',
                    'elapsed time:timestamp - lag_6',
                    'elapsed time:timestamp - lag_7',
                    'elapsed time:timestamp - lag_8',
                    'elapsed time:timestamp - lag_9',
                    'elapsed time:timestamp - lag_10',
                    'elapsed time:timestamp - lag_11',
                    'elapsed time:timestamp - lag_12',
                    'elapsed time:timestamp - lag_13',
                    'elapsed time:timestamp - lag_14',
                    'elapsed time:timestamp - lag_15',
                    'elapsed time:timestamp - lag_16',
                    'elapsed time:timestamp - lag_17',
                    'elapsed time:timestamp - lag_18',
                    'elapsed time:timestamp - lag_19',
                    'elapsed time:timestamp - lag_20',
                    'elapsed time:timestamp - lag_21',
                    'elapsed time:timestamp - lag_22',
                    'elapsed time:timestamp - lag_23',
                    'elapsed time:timestamp - lag_24',
                    'elapsed time:timestamp - lag_25',
                    'elapsed time:timestamp - lag_26',
                    'elapsed time:timestamp - lag_27',
                    'elapsed time:timestamp - lag_28',
                    'elapsed time:timestamp - lag_29',
                    'elapsed time:timestamp - lag_30']]

X_train = df_train[['concept:name', 
                    'concept:name - lag_1', 
                    'concept:name - lag_2', 
                    'concept:name - lag_3', 
                    'next concept:name rfc', 
                    'year', 
                    'month', 
                    'day', 
                    'hour',
                    'CreditScore', 
                    'org:resource',
                    'elapsed time:timestamp - lag_1',
                    'elapsed time:timestamp - lag_2',
                    'elapsed time:timestamp - lag_3',
                    'elapsed time:timestamp - lag_4',
                    'elapsed time:timestamp - lag_5',
                    'elapsed time:timestamp - lag_6',
                    'elapsed time:timestamp - lag_7',
                    'elapsed time:timestamp - lag_8',
                    'elapsed time:timestamp - lag_9',
                    'elapsed time:timestamp - lag_10',
                    'elapsed time:timestamp - lag_11',
                    'elapsed time:timestamp - lag_12',
                    'elapsed time:timestamp - lag_13',
                    'elapsed time:timestamp - lag_14',
                    'elapsed time:timestamp - lag_15',
                    'elapsed time:timestamp - lag_16',
                    'elapsed time:timestamp - lag_17',
                    'elapsed time:timestamp - lag_18',
                    'elapsed time:timestamp - lag_19',
                    'elapsed time:timestamp - lag_20',
                    'elapsed time:timestamp - lag_21',
                    'elapsed time:timestamp - lag_22',
                    'elapsed time:timestamp - lag_23',
                    'elapsed time:timestamp - lag_24',
                    'elapsed time:timestamp - lag_25',
                    'elapsed time:timestamp - lag_26',
                    'elapsed time:timestamp - lag_27',
                    'elapsed time:timestamp - lag_28',
                    'elapsed time:timestamp - lag_29',
                    'elapsed time:timestamp - lag_30']]

X_test = df_test[['concept:name', 
                    'concept:name - lag_1', 
                    'concept:name - lag_2', 
                    'concept:name - lag_3', 
                    'next concept:name rfc', 
                    'year', 
                    'month', 
                    'day', 
                    'hour',
                    'CreditScore',
                    'org:resource',
                    'elapsed time:timestamp - lag_1', 
                    'elapsed time:timestamp - lag_2', 
                    'elapsed time:timestamp - lag_3',
                    'elapsed time:timestamp - lag_4',
                    'elapsed time:timestamp - lag_5',
                    'elapsed time:timestamp - lag_6',
                    'elapsed time:timestamp - lag_7',
                    'elapsed time:timestamp - lag_8',
                    'elapsed time:timestamp - lag_9',
                    'elapsed time:timestamp - lag_10',
                    'elapsed time:timestamp - lag_11',
                    'elapsed time:timestamp - lag_12',
                    'elapsed time:timestamp - lag_13',
                    'elapsed time:timestamp - lag_14',
                    'elapsed time:timestamp - lag_15',
                    'elapsed time:timestamp - lag_16',
                    'elapsed time:timestamp - lag_17',
                    'elapsed time:timestamp - lag_18',
                    'elapsed time:timestamp - lag_19',
                    'elapsed time:timestamp - lag_20',
                    'elapsed time:timestamp - lag_21',
                    'elapsed time:timestamp - lag_22',
                    'elapsed time:timestamp - lag_23',
                    'elapsed time:timestamp - lag_24',
                    'elapsed time:timestamp - lag_25',
                    'elapsed time:timestamp - lag_26',
                    'elapsed time:timestamp - lag_27',
                    'elapsed time:timestamp - lag_28',
                    'elapsed time:timestamp - lag_29',
                    'elapsed time:timestamp - lag_30']]

y_train = df_train['elapsed time:timestamp']

y_test = df_test[['elapsed time:timestamp']]

print(f"""
    inputs: {[col for col in X_test.columns]} \n
    target: {[col for col in y_test.columns]}
""")
# Define the parameter grid

param_grid = {
    'n_estimators': [85],
    'max_depth': [7],
    'learning_rate': [0.099]
}
# Initialize the model
model = XGBRegressor(objective='reg:squarederror')

# Initialize GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the GridSearchCV using the transformed training set
grid_search.fit(X_train, y_train)

# Get the best parameters and best estimator
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print(f"best params: {best_params}")

# Save the best model
model_filename = '../model_weights/xgboost.joblib'
dump(model, model_filename)
        
# Predict on the test set using the best estimator
y_pred_test = np.abs(best_estimator.predict(X_test))

# Evaluate the model on the test set
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred_test))
print(f'Test RMSE: {rmse_test}')

r2_score_value = r2_score(y_test, y_pred_test)
print(f'R² score: {r2_score_value}')

# Update the dataframe with predictions from the best model
df['elapsed time:timestamp XGBoost'] = np.abs(np.where(best_estimator.predict(X) < 0, np.abs(best_estimator.predict(X)) / 10000, best_estimator.predict(X) / 1000))
df['elapsed time:timestamp'] = df['elapsed time:timestamp'].mask(df['elapsed time:timestamp'] < 0)
df = df.drop(columns=['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0', 'year', 'month', 'day', 'hour', 'elapsed time:timestamp - lag_10', 'elapsed time:timestamp - lag_9', 'elapsed time:timestamp - lag_8', 'elapsed time:timestamp - lag_7', 'elapsed time:timestamp - lag_6', 'elapsed time:timestamp - lag_5', 'elapsed time:timestamp - lag_4', 'elapsed time:timestamp - lag_3', 'elapsed time:timestamp - lag_2', 'elapsed time:timestamp - lag_1', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3',
                    'elapsed time:timestamp - lag_11',
                    'elapsed time:timestamp - lag_12',
                    'elapsed time:timestamp - lag_13',
                    'elapsed time:timestamp - lag_14',
                    'elapsed time:timestamp - lag_15',
                    'elapsed time:timestamp - lag_16',
                    'elapsed time:timestamp - lag_17',
                    'elapsed time:timestamp - lag_18',
                    'elapsed time:timestamp - lag_19',
                    'elapsed time:timestamp - lag_20',
                    'elapsed time:timestamp - lag_21',
                    'elapsed time:timestamp - lag_22',
                    'elapsed time:timestamp - lag_23',
                    'elapsed time:timestamp - lag_24',
                    'elapsed time:timestamp - lag_25',
                    'elapsed time:timestamp - lag_26',
                    'elapsed time:timestamp - lag_27',
                    'elapsed time:timestamp - lag_28',
                    'elapsed time:timestamp - lag_29',
                    'elapsed time:timestamp - lag_30'], errors='ignore')
df['elapsed time:timestamp'] = pd.to_timedelta(df['elapsed time:timestamp'], unit='seconds')
df['elapsed time:timestamp XGBoost'] = pd.to_timedelta(df['elapsed time:timestamp XGBoost'], unit='seconds')
df['next time:timestamp XGBoost'] = df['time:timestamp'] + df['elapsed time:timestamp XGBoost']
df = df.drop(columns=['elapsed time:timestamp XGBoost', 'elapsed time:timestamp'], errors='ignore')

# Decode categorical columns
for column in df.columns:
    if column in ['concept:name', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3', 'next concept:name rfc', 'org:resource']:
        le = preprocessors[column]
        df[column] = le.inverse_transform(df[column])
    else:
        continue 

df.to_csv(f'../data/{chosed_dataset}_rfc_xgboost.csv')

Train set ends with 2016-10-27 09:12:04.286000
Test set starts with: 2016-10-27 09:26:15.278000

    inputs: ['concept:name', 'concept:name - lag_1', 'concept:name - lag_2', 'concept:name - lag_3', 'next concept:name rfc', 'year', 'month', 'day', 'hour', 'CreditScore', 'org:resource', 'elapsed time:timestamp - lag_1', 'elapsed time:timestamp - lag_2', 'elapsed time:timestamp - lag_3', 'elapsed time:timestamp - lag_4', 'elapsed time:timestamp - lag_5', 'elapsed time:timestamp - lag_6', 'elapsed time:timestamp - lag_7', 'elapsed time:timestamp - lag_8', 'elapsed time:timestamp - lag_9', 'elapsed time:timestamp - lag_10', 'elapsed time:timestamp - lag_11', 'elapsed time:timestamp - lag_12', 'elapsed time:timestamp - lag_13', 'elapsed time:timestamp - lag_14', 'elapsed time:timestamp - lag_15', 'elapsed time:timestamp - lag_16', 'elapsed time:timestamp - lag_17', 'elapsed time:timestamp - lag_18', 'elapsed time:timestamp - lag_19', 'elapsed time:timestamp - lag_20', 'elapsed time:timestamp

In [20]:
df

,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,case:concept:name,...,OfferedAmount,OfferID,start_timestamp,time:timestamp diff,next time:timestamp,next time:timestamp naive,next concept:name naive,next concept:name,next concept:name rfc,next time:timestamp XGBoost
0,Created,User_1,A_Create Application,Application,Application_652823628,complete,2016-01-01 09:51:15.304,Existing loan takeover,New credit,652823628,...,NaN,NaN,2016-01-01 09:51:15.304000+00:00,NaN,2016-01-01 09:51:15.352,2016-01-01 09:51:15.344,A_Submitted,A_Submitted,A_Submitted,2016-01-01 09:51:15.687974522
1,statechange,User_1,A_Submitted,Application,ApplState_1582051990,complete,2016-01-01 09:51:15.352,Existing loan takeover,New credit,652823628,...,NaN,NaN,2016-01-01 09:51:15.352000+00:00,0 days 00:00:00.048000,2016-01-01 09:51:15.774,2016-01-01 09:51:15.652,W_Handle leads,W_Handle leads,W_Handle leads,2016-01-01 09:51:16.067824127
2,Created,User_1,W_Handle leads,Workflow,Workitem_1298499574,schedule,2016-01-01 09:51:15.774,Existing loan takeover,New credit,652823628,...,NaN,NaN,2016-01-01 09:51:15.774000+00:00,0 days 00:00:00.422000,2016-01-01 09:52:36.392,2016-01-01 09:51:49.854,W_Handle leads,W_Handle leads,W_Handle leads,2016-01-01 09:51:55.005128693
3,Deleted,User_1,W_Handle leads,Workflow,Workitem_1673366067,withdraw,2016-01-01 09:52:36.392,Existing loan takeover,New credit,652823628,...,NaN,NaN,2016-01-01 09:52:36.392000+00:00,0 days 00:01:20.618000,2016-01-01 09:52:36.403,2016-01-01 09:53:10.472,W_Complete application,W_Complete application,W_Complete application,2016-01-01 09:52:37.237262766
4,Created,User_1,W_Complete application,Workflow,Workitem_1493664571,schedule,2016-01-01 09:52:36.403,Existing loan takeover,New credit,652823628,...,NaN,NaN,2016-01-01 09:52:36.403000+00:00,0 days 00:00:00.011000,2016-01-01 09:52:36.413,2016-01-01 09:52:40.113,A_Concept,A_Concept,A_Concept,2016-01-01 09:52:36.790111843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201085,statechange,User_131,A_Validating,Application,ApplState_1576288280,complete,2017-02-01 14:00:30.347,Existing loan takeover,New credit,637536789,...,NaN,NaN,2017-02-01 14:00:30.347000+00:00,0 days 00:00:00.072000,2017-02-01 14:01:24.191,2017-02-01 14:00:56.427,W_Call incomplete files,W_Validate application,W_Shortened completion,2017-02-01 14:01:09.789512512
1201086,Released,User_131,W_Validate application,Workflow,Workitem_997581455,suspend,2017-02-01 14:01:24.191,Existing loan takeover,New credit,637536789,...,NaN,NaN,2017-02-01 14:01:24.191000+00:00,0 days 00:00:53.844000,NaN,2017-02-01 14:01:25.031,NaN,NaN,W_Shortened completion,2017-02-01 14:01:56.495859161
1201087,Released,User_56,W_Call incomplete files,Workflow,Workitem_1421607593,suspend,2017-02-01 14:09:03.331,"Other, see explanation",New credit,16193774,...,NaN,NaN,2017-02-01 14:09:03.331000+00:00,0 days 00:18:06.767000,NaN,2017-02-01 14:10:13.841,NaN,NaN,W_Call incomplete files,2017-02-01 14:09:26.709137589
1201088,Obtained,User_100,W_Validate application,Workflow,Workitem_1317562192,resume,2017-02-01 14:10:52.793,Existing loan takeover,New credit,586083090,...,NaN,NaN,2017-02-01 14:10:52.793000+00:00,0 days 00:16:49.249000,2017-02-01 14:11:03.499,2017-02-01 14:10:53.633,W_Call incomplete files,W_Validate application,W_Shortened completion,2017-02-01 14:11:13.945019501


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plotting feature importances
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

plt.title('Feature Importance', fontsize=20)
plt.xlabel('Importance', fontsize=18)
plt.ylabel('Feature', fontsize=18)

# Increase tick label size
barplot.xaxis.set_tick_params(labelsize=16)
barplot.yaxis.set_tick_params(labelsize=16)

plt.show()
plt.savefig('feature.png')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Convert the importances into a DataFrame
feature_importance_df = pd.DataFrame({
    'Feature': X.columns,
    'Importance': importances
}).sort_values(by='Importance', ascending=True)

# Plotting feature importances
plt.figure(figsize=(12, 8))
barplot = sns.barplot(x='Importance', y='Feature', data=feature_importance_df)

plt.title('Feature Importance', fontsize=20)
plt.xlabel('Importance', fontsize=18)
plt.ylabel('Feature', fontsize=18)

# Increase tick label size
barplot.xaxis.set_tick_params(labelsize=16)
barplot.yaxis.set_tick_params(labelsize=16)

plt.show()
plt.savefig('feature.png')

In [ ]:
predicted_counts = df_2017['concept:name next rfc'].value_counts()

plt.figure(figsize=(10, 7))
sns.barplot(x=predicted_counts.index, y=predicted_counts.values)
plt.title('Predicted Next Step Distribution')
plt.xlabel('Next Steps')
plt.ylabel('Frequency')
plt.xticks(rotation=45)
plt.show()

In [ ]:
class_names = label_encoder.inverse_transform(range(n_classes))

# Compute ROC curve and ROC area for each class
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binarized[:, i], y_pred_proba[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plotting
plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], label=f'ROC curve of {class_names[i]} (area = {roc_auc[i]:.2f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve with True Class Names')
plt.legend(loc="lower right", prop={'size': 6})  # Adjust size as needed
plt.show()


In [ ]:
# Compute precision-recall curve and average precision for each class
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(y_test_binarized[:, i],
                                                        y_pred_proba[:, i])
    average_precision[i] = average_precision_score(y_test_binarized[:, i], y_pred_proba[:, i])

# Plotting
plt.figure(figsize=(10, 7))
for i in range(n_classes):
    plt.plot(recall[i], precision[i], label=f'{class_names[i]} (area = {average_precision[i]:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve with True Class Names')
plt.legend(loc="best", prop={'size': 6})  # Adjust size as needed
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Assume 'average_precision' is a dictionary containing the AP scores for each class
# where keys are numerical class indices and values are AP scores.

# Get class names from the label encoder
class_names = label_encoder.inverse_transform(sorted(average_precision.keys()))

# Get AP scores in the order of class names
ap_scores = [average_precision[i] for i in sorted(average_precision.keys())]

# Create a bar plot
plt.figure(figsize=(12, 8))
bars = plt.bar(range(len(class_names)), ap_scores, color='skyblue')

# Add the AP scores above the bars
for bar in bars:
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2.0, yval, round(yval, 2), va='bottom')  # va: vertical alignment

plt.xlabel('Class Names')
plt.ylabel('Average Precision Score')
plt.title('Average Precision Scores by Class')
plt.xticks(range(len(class_names)), class_names, rotation=90)  # Rotate class names for better visibility
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Assuming 'y_test' are your true labels and 'y_pred' are your predicted labels
# If 'y_pred' is not yet defined, you would need to predict it using your model
y_pred = best_model.predict(X_test)

# Calculate the confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Assuming 'label_encoder' is the LabelEncoder instance used to encode your labels
# Use the inverse transform method to get the original class names
class_names = label_encoder.inverse_transform(np.arange(cm.shape[0]))

# Calculate error rates for each class
error_rate = (cm.sum(axis=1) - np.diag(cm)) / cm.sum(axis=1)

# Plot the error rate by class
plt.figure(figsize=(12, 8))
plt.bar(class_names, error_rate, color='salmon')
plt.xlabel('Class Names')
plt.ylabel('Error Rate')
plt.title('Error Rate by Class')
plt.xticks(rotation=90)  # Rotate the class names for better readability if needed
plt.show()



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Generate the classification report
report = classification_report(y_test, y_pred, output_dict=True)

# Convert the report to a DataFrame
report_df = pd.DataFrame(report).transpose()

# Replace index with class names for clarity, using label_encoder
class_names = label_encoder.inverse_transform(sorted(np.unique(y_test)))

# Exclude the 'accuracy' row and split the DataFrame into metrics and support
metrics_df = report_df.iloc[:-3, :-1]  # Exclude the 'support' column and the last 3 summary rows
support_df = report_df.iloc[:-3, -1:].astype(int)  # Keep only the 'support' column, convert to int
metrics_df.index = class_names
support_df.index = class_names

# Setup the matplotlib figure and axes for a side-by-side plot
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 10), gridspec_kw={'width_ratios': [3, 1]})

# Plot the heatmap for metrics
sns.heatmap(metrics_df, annot=True, fmt=".2f", cmap='Blues', ax=ax1, cbar=False, annot_kws={'size':18})
ax1.set_title('Precision, Recall, and F1-Score', fontsize=20)
ax1.set_ylabel('Classes', fontsize=18)
ax1.set_xlabel('Metrics', fontsize=18)
ax1.tick_params(axis='both', which='major', labelsize=16)

# Plot the heatmap (actually just annotated cells) for support
sns.heatmap(support_df, annot=True, fmt="d", cmap=None, ax=ax2, cbar=False, annot_kws={'size':18})
ax2.set_title('Support', fontsize=20)
ax2.set_xlabel('Metrics', fontsize=18)
ax2.tick_params(axis='y', which='major', labelsize=16)
ax2.set_yticks([]) 

plt.tight_layout()
plt.show()
plt.savefig('Precision_recall_F1_support_scores.jpg')

### Only for test purposes, do not run it

### Use this sample of code for future in Sprint 3

In [ ]:
# create dataset
X = df_2017[['concept:name - lag_2', 'concept:name - lag_1', 'concept:name']]
y = df_2017[['concept:name next']]

# Use LabelEncoder to convert string labels to numerical values
label_encoder = LabelEncoder()
X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
X['concept:name'] = label_encoder.fit_transform(X['concept:name'])
y = label_encoder.fit_transform(y)

# configure the cross-validation procedure
cv_outer = KFold(n_splits=10, shuffle=False, random_state=None)
# enumerate splits
outer_results = list()
for train_ix, test_ix in cv_outer.split(X):
    # split data
    X_train, X_test = X.iloc[train_ix, :], X.iloc[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # configure the cross-validation procedure
    cv_inner = KFold(n_splits=3, shuffle=False, random_state=None)
    # define the model
    model = RandomForestClassifier(n_jobs=1)
    # define search
    search = GridSearchCV(model, param_grid=param_grid, scoring='accuracy', cv=cv_inner, refit=True)
    # execute search
    result = search.fit(X_train, y_train)
    # get the best performing model fit on the whole training set
    best_model = result.best_estimator_
    # evaluate model on the hold out dataset
    yhat = best_model.predict(X_test)
    # evaluate the model
    acc = accuracy_score(y_test, yhat)
    # store the result
    outer_results.append(acc)
    # report progress
    print('>acc=%.3f, est=%.3f, cfg=%s' % (acc, result.best_score_, result.best_params_))
# summarize the estimated performance of the model
print('Accuracy: %.3f (%.3f)' % (mean(outer_results), std(outer_results)))

/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_1'] = label_encoder.fit_transform(X['concept:name - lag_1'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['concept:name - lag_2'] = label_encoder.fit_transform(X['concept:name - lag_2'])
/var/folders/22/gxvfn3cx2_v64ngzhq52h2180000gn/T/ipykernel_9386/504684746.py:9: SettingWithCop

>acc=0.980, est=0.950, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 2, 'min_samples_split': 10, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 19}
>acc=0.950, est=0.953, cfg={'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 21}
>acc=0.960, est=0.952, cfg={'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 19}
>acc=0.960, est=0.952, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 20}
>acc=0.940, est=0.954, cfg={'max_depth': 18, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 21}
>acc=0.920, est=0.957, cfg={'max_depth': 19, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 20}
>acc=0.919, est=0.955, cfg={'max_depth': 18, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimat

In [ ]:
# Save the best model to a file
model_filename = 'random_forest_model.joblib'
joblib.dump(best_model, model_filename)

# Save the best model to a file using pickle
model_filename = 'random_forest_model.pkl'
with open(model_filename, 'wb') as model_file:
    pickle.dump(best_model, model_file)

In [ ]:
df_2017

,Unnamed: 0,Action,org:resource,concept:name,EventOrigin,EventID,lifecycle:transition,time:timestamp,case:LoanGoal,case:ApplicationType,...,predicted_time_to_next_event_seconds,elapsed_time_from_start,predicted_start_time,predicted_step,predicted_time_to_next_event_seconds_XG,concept:name - lag_1,concept:name - lag_2,concept:name - lag_3,concept:name next,predicted_next_event_rfc
3,3.0,1,0,22,2,701704,6,3,5,1,...,0 days 00:00:35.170000,33119,128,7,0 days 00:00:15,22.0,8.0,4.0,21.0,21
4,4.0,0,0,21,2,630431,3,4,5,1,...,0 days 00:00:00.010000,33126,127,6,0 days 00:00:05.500000,22.0,22.0,8.0,3.0,3
5,5.0,4,0,3,0,169679,1,5,5,1,...,2 days 01:50:47.740000,33133,126,0,4 days 22:54:39.812500,21.0,22.0,22.0,4.0,4
6,6.0,0,0,4,0,219156,1,6,7,1,...,0 days 00:00:00.040000,0,112,1,0 days 00:00:53.700001,3.0,21.0,22.0,8.0,8
7,7.0,4,0,8,0,130913,1,7,7,1,...,0 days 00:00:00.300000,49,113,3,0 days 00:00:11.200000,4.0,3.0,21.0,22.0,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1201085,1201085.0,1,148,25,2,708416,0,1200970,11,1,...,0 days 00:00:00.020000,999899,44,5,0 days 00:01:00.799999,7.0,10.0,20.0,11.0,25
1201086,1201086.0,4,148,11,1,323681,1,1200971,11,1,...,0 days 00:00:00.010000,999900,45,5,0 days 00:01:17.500000,25.0,7.0,10.0,11.0,25
1201087,1201087.0,4,148,11,1,250417,1,1200972,11,1,...,0 days 00:00:00.010000,999901,43,5,0 days 00:01:17.500000,11.0,25.0,7.0,11.0,25
1201088,1201088.0,4,148,11,1,316489,1,1200973,11,1,...,0 days 00:00:00.010000,999902,47,5,0 days 00:01:17.500000,11.0,11.0,25.0,11.0,19
